In [0]:
%sql

SHOW TABLES IN samples.bakehouse;

We are intereted in `sales_transactions` table to see if machine learing can predict the sales in the future. 

In [0]:
DESCRIBE TABLE samples.bakehouse.sales_transactions;

Let us see how many records of data we have. 

In [0]:
SELECT COUNT(*) AS total_rows FROM samples.bakehouse.sales_transactions

3333 lines of data.

In [0]:
SELECT 
  MIN(dateTime) AS min_ts,
  MAX(dateTime) AS max_ts
FROM samples.bakehouse.sales_transactions

It is about 2 and half weeks of data, starting on the day 2024 May 1. Let us see the actual data, say, first 10 of them.  

In [0]:
SELECT *
  FROM samples.bakehouse.sales_transactions
  ORDER BY dateTime
  LIMIT 10


The table is normalized. Should I join? -> at the moment, go without. Let us NULL for the main columns. 

In [0]:
SELECT 
  SUM(CASE WHEN dateTime IS NULL THEN 1 ELSE 0 END) AS null_dateTime,
  SUM(CASE WHEN franchiseID IS NULL THEN 1 ELSE 0 END) AS null_franchiseID,
  SUM(CASE WHEN product IS NULL THEN 1 ELSE 0 END) AS null_product,
  SUM(CASE WHEN quantity IS NULL THEN 1 ELSE 0 END) AS null_quantity,
  SUM(CASE WHEN totalPrice IS NULL THEN 1 ELSE 0 END) AS null_totalPrice
FROM samples.bakehouse.sales_transactions

No NULL values were found.
Next, we aggregate the sales by date.

In [0]:
WITH daily AS (
  SELECT 
    DATE(dateTime) AS ds,
    SUM(totalPrice) AS y
  FROM samples.bakehouse.sales_transactions
  GROUP BY DATE(dateTime)
)
SELECT * 
FROM daily
ORDER BY ds;


```WITH daily AS``` is called `common table expression` (CTE). 
We will enrich the table with the day of week, the total sales on one day before,  the total sales on one week before, and a moving average over a week.


In [0]:
WITH daily AS (
  SELECT
    DATE(dateTime) AS ds,
    SUM(totalPrice) AS y
  FROM
    samples.bakehouse.sales_transactions
  GROUP BY
    DATE(dateTime)
),
features AS (
  SELECT
    ds,
    y,
    dayofweek(ds) AS dow,
    LAG(y, 1) OVER (ORDER BY ds) AS lag_1,
    LAG(y, 7) OVER (ORDER BY ds) AS lag_7,
    AVG(y) OVER (ORDER BY ds ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma_7
  FROM
    daily
)
SELECT
  *
FROM
  features
ORDER BY
  ds

Plot sales over the ones 7 days before.  